In [5]:
import copy
import random
import numpy as np

class RiverSwim:
    def __init__(self, current, seed=1234):
        self.num_states = 6
        self.num_actions = 2  # O <=> LEFT, 1 <=> RIGHT

        # Larger current makes it harder to swim up the river
        self.currents = ['WEAK', 'MEDIUM', 'STRONG']
        assert current in self.currents
        self.current = self.currents.index(current) + 1
        assert self.current in [1, 2, 3]

        # Configure reward function
        R = np.zeros((self.num_states, self.num_actions))
        R[0, 0] = 0.005
        R[5, 1] = 1.

        # Configure transition function
        T = np.zeros((self.num_states, self.num_actions, self.num_states))

        # Encode initial and rewarding state transitions
        T[0, 0, 0] = 1.
        T[0, 1, 0] = 0.6
        T[0, 1, 1] = 0.4

        T[5, 1, 5] = 0.6
        T[5, 1, 4] = 0.4
        T[5, 0, 4] = 1.

        # Encode intermediate state transitions
        for s in range(1, self.num_states - 1):
            left, right = 0, 1

            # Going left always succeeds
            T[s, left, s - 1] = 1.

            # Going right sometimes succeeds
            T[s, right, s] = 0.6
            T[s, right, s - 1] = 0.09 * self.current
            T[s, right, s + 1] = 0.4 - T[s, right, s - 1]
            assert np.isclose(np.sum(T[s, right]), 1.)

        self.R = np.array(R)
        self.T = np.array(T)

        # Agent always starts at the opposite end of the river
        self.init_state = 0
        self.curr_state = self.init_state

        self.seed = seed
        random.seed(self.seed)
        np.random.seed(self.seed)

    def get_model(self):
        return copy.deepcopy(self.R), copy.deepcopy(self.T)

    def reset(self):
        return self.init_state

    def step(self, action):
        reward = self.R[self.curr_state, action]
        next_state = np.random.choice(range(self.num_states), p=self.T[self.curr_state, action])
        self.curr_state = next_state
        return reward, next_state

In [8]:
np.set_printoptions(precision=3)

def bellman_backup(state, action, R, T, gamma, V):
    """
    Perform a single Bellman backup.

    Parameters
    ----------
    state: int
    action: int
    R: np.array (num_states, num_actions)
    T: np.array (num_states, num_actions, num_states)
    gamma: float
    V: np.array (num_states)

    Returns
    -------
    backup_val: float
    """
    backup_val = 0.
    ############################
    # YOUR IMPLEMENTATION HERE #
    for next_state in range(len(V)):
        backup_val += T[state, action, next_state] * (R[state, action] + gamma * V[next_state])
    ############################

    return backup_val

def policy_evaluation(policy, R, T, gamma, tol=1e-3):
    """
    Compute the value function induced by a given policy for the input MDP
    Parameters
    ----------
    policy: np.array (num_states)
    R: np.array (num_states, num_actions)
    T: np.array (num_states, num_actions, num_states)
    gamma: float
    tol: float

    Returns
    -------
    value_function: np.array (num_states)
    """
    num_states, num_actions = R.shape
    value_function = np.zeros(num_states)

    ############################
    # YOUR IMPLEMENTATION HERE #

    while True:
        delta = 0
        for s in range(num_states):
            v = value_function[s]
            action = policy[s]
            value_function[s] = bellman_backup(s, action, R, T, gamma, value_function)
            delta = max(delta, abs(v - value_function[s]))
        if delta < tol:
            break

    ############################
    return value_function


def policy_improvement(policy, R, T, V_policy, gamma):
    """
    Given the value function induced by a given policy, perform policy improvement
    Parameters
    ----------
    policy: np.array (num_states)
    R: np.array (num_states, num_actions)
    T: np.array (num_states, num_actions, num_states)
    V_policy: np.array (num_states)
    gamma: float

    Returns
    -------
    new_policy: np.array (num_states)
    """
    num_states, num_actions = R.shape
    new_policy = np.zeros(num_states, dtype=int)

    ############################
    # YOUR IMPLEMENTATION HERE #

    for s in range(num_states):
        action_values = np.zeros(num_actions)
        for a in range(num_actions):
            action_values[a] = bellman_backup(s, a, R, T, gamma, V_policy)
        new_policy[s] = np.argmax(action_values)
    ############################
    return new_policy


def policy_iteration(R, T, gamma, tol=1e-3):
    """Runs policy iteration.

    You should call the policy_evaluation() and policy_improvement() methods to
    implement this method.
    Parameters
    ----------
    R: np.array (num_states, num_actions)
    T: np.array (num_states, num_actions, num_states)

    Returns
    -------
    V_policy: np.array (num_states)
    policy: np.array (num_states)
    """
    num_states, num_actions = R.shape
    V_policy = np.zeros(num_states)
    policy = np.zeros(num_states, dtype=int)
    ############################
    # YOUR IMPLEMENTATION HERE #
    is_policy_stable = False
    while not is_policy_stable:
        V_policy = policy_evaluation(policy, R, T, gamma) # calculate value function in current policy
        new_policy = policy_improvement(policy, R, T, V_policy, gamma) # improve policy based on value function
        is_policy_stable = np.all(policy == new_policy)
        policy = new_policy
    ############################
    return V_policy, policy


def value_iteration(R, T, gamma, tol=1e-3):
    """Runs value iteration.
    Parameters
    ----------
    R: np.array (num_states, num_actions)
    T: np.array (num_states, num_actions, num_states)

    Returns
    -------
    value_function: np.array (num_states)
    policy: np.array (num_states)
    """
    num_states, num_actions = R.shape
    value_function = np.zeros(num_states)
    policy = np.zeros(num_states, dtype=int)
    ############################
    # YOUR IMPLEMENTATION HERE #

    while True:
        delta = 0
        for s in range(num_states):
            v = value_function[s]
            action_values = np.zeros(num_actions)
            for a in range(num_actions):
                action_values[a] = bellman_backup(s, a, R, T, gamma, value_function)
            value_function[s] = np.max(action_values)
            policy[s] = np.argmax(action_values)
            delta = max(delta, abs(v - value_function[s]))
        if delta < tol:
            break

            

    ############################
    return value_function, policy

In [3]:
# Edit below to run policy and value iteration on different configurations
# You may change the parameters in the functions below
SEED = 1234

RIVER_CURRENT = 'WEAK'
assert RIVER_CURRENT in ['WEAK', 'MEDIUM', 'STRONG']
env = RiverSwim(RIVER_CURRENT, SEED)

R, T = env.get_model()
discount_factor = 0.99

In [7]:
# print("\n" + "-" * 25 + "\nBeginning Policy Iteration\n" + "-" * 25)

V_pi, policy_pi = policy_iteration(R, T, gamma=discount_factor, tol=1e-3)
print(V_pi)
print([['L', 'R'][a] for a in policy_pi])


[30.348 31.116 32.356 33.774 35.288 36.881]
['R', 'R', 'R', 'R', 'R', 'R']


In [9]:
# print("\n" + "-" * 25 + "\nBeginning Value Iteration\n" + "-" * 25)

V_vi, policy_vi = value_iteration(R, T, gamma=discount_factor, tol=1e-3)
print(V_vi)
print([['L', 'R'][a] for a in policy_vi])

[30.348 31.116 32.356 33.774 35.288 36.881]
['R', 'R', 'R', 'R', 'R', 'R']
